In [ ]:
# My OpenAI Key
import os
os.environ['OPENAI_API_KEY'] = "INSERT OPENAI KEY"

In [ ]:
from gpt_index import GPTTreeIndex, SimpleDirectoryReader
from IPython.display import Markdown, display

In [ ]:
documents = SimpleDirectoryReader('data').load_data()
index = GPTTreeIndex(documents)

In [ ]:
index.save_to_disk('index.json')

In [ ]:
# try loading
new_index = GPTTreeIndex.load_from_disk('index.json', child_branch_factor=1)

In [ ]:
# try verbose=True for more detailed outputs
response = new_index.query("What did the author do growing up?")

In [ ]:
display(Markdown(f"<b>{response}</b>"))

In [ ]:
# try using branching factor 2
new_index_b2 = GPTTreeIndex.load_from_disk('index.json', child_branch_factor=2)
response_b2 = new_index_b2.query("What did the author do growing up?")

In [ ]:
display(Markdown(f"<b>{response_b2}</b>"))

In [ ]:
# try verbose=True for more detailed outputs
new_index.query("What did the author do after his time at Y Combinator?")